# Finding Available Rates

We can get a single library with all the rates known to pynucastro---this is a combination of {py:obj}`ReacLibLibrary <pynucastro.rates.library.ReacLibLibrary>`, all of the different collections that can be part of {py:obj}`TabularLibrary <pynucastro.rates.library.TabularLibrary>`, and the rates contained in {py:mod}`alternate_rates <pynucastro.rates.alternate_rates>`.  This is done via {py:meth}`full_library <pynucastro.rates.library.full_library>`.

```{note}
The {py:obj}`Library <pynucastro.rates.library.Library>` returned by `full_library` will have duplicates for some links, since multiple sources can provide a rate.
```

In [1]:
import pynucastro as pyna

In [2]:
lib = pyna.full_library()

We can see the total number of rates

In [3]:
lib.num_rates

82491

## Using filters

We can then use a {py:obj}`RateFilter <pynucastro.rates.library.RateFilter>` to find rates.

For example, to find all the rates that involve ${}^{12}\mathrm{C}$ as a reactant, we can do:

In [4]:
rf = pyna.RateFilter(reactants=["c12"], exact=False)
rates = lib.filter(rf).get_rates()

We can look at the source from the literature and the pynucastro class for each of the rates:

In [5]:
for r in sorted(rates):
    print(f"{str(r):24} {r.source['Label']:6}  {type(r).__name__}")

C12 + n ⟶ He4 + Be9      cf88    ReacLibRate
C12 + n ⟶ p + B12        wag     ReacLibRate
C12 + n ⟶ C13 + 𝛾        ks03    ReacLibRate
C12 + p ⟶ N13 + 𝛾        ls09    ReacLibRate
C12 + He4 ⟶ p + N15      nacr    ReacLibRate
C12 + He4 ⟶ n + O15      cf88    ReacLibRate
C12 + He4 ⟶ O16 + 𝛾      debo    DeBoerC12agO16
C12 + He4 ⟶ O16 + 𝛾      nac2    ReacLibRate
C12 + C12 ⟶ He4 + Ne20   cf88    ReacLibRate
C12 + C12 ⟶ p + Na23     cf88    ReacLibRate
C12 + C12 ⟶ n + Mg23     cf88    ReacLibRate
C12 ⟶ 3 He4              fy05    ReacLibRate
C12 ⟶ p + B11            nw00    ReacLibRate
C12 ⟶ n + C11            bb92    ReacLibRate
O16 + C12 ⟶ He4 + Mg24   cf88    ReacLibRate
O16 + C12 ⟶ p + Al27     cf88    ReacLibRate
O16 + C12 ⟶ n + Si27     cf88    ReacLibRate
Ne20 + C12 ⟶ He4 + Si28  rolf    ReacLibRate
Ne20 + C12 ⟶ p + P31     rolf    ReacLibRate
Ne20 + C12 ⟶ n + S31     rolf    ReacLibRate


Here we see that there are 2 sources for ${}^{12}\mathrm{C}(\alpha,\gamma){}^{16}\mathrm{O}$: NACRE and DeBoer et al.

What about ${}^{56}\mathrm{Ni}$?

In [8]:
rf = pyna.RateFilter(reactants=["ni56"], exact=False)
new_lib = lib.filter(rf)
rates = new_lib.get_rates()

In [9]:
for r in sorted(rates):
    print(f"{str(r):24} {r.source['Label']:10}  {type(r).__name__}")

Ni56 + e⁻ ⟶ Co56 + 𝜈     ffn         TabularRate
Ni56 + e⁻ ⟶ Co56 + 𝜈     langanke    TabularRate
Ni56 + n ⟶ He4 + Fe53    ths8        ReacLibRate
Ni56 + n ⟶ p + Co56      ths8        ReacLibRate
Ni56 + n ⟶ Ni57 + 𝛾      ths8        ReacLibRate
Ni56 + p ⟶ He4 + Co53    ths8        ReacLibRate
Ni56 + p ⟶ n + Cu56      ths8        ReacLibRate
Ni56 + p ⟶ Cu57 + 𝛾      wien        ReacLibRate
Ni56 + He4 ⟶ p + Cu59    ths8        ReacLibRate
Ni56 + He4 ⟶ n + Zn59    ths8        ReacLibRate
Ni56 + He4 ⟶ Zn60 + 𝛾    ths8        ReacLibRate
Ni56 ⟶ He4 + Fe52        ths8        ReacLibRate
Ni56 ⟶ p + Co55          ths8        ReacLibRate
Ni56 ⟶ Co56 + e⁺ + 𝜈     wc12        ReacLibRate
Ni56 ⟶ n + Ni55          ths8        ReacLibRate


Now we see that there are three electron capture / $\beta^+$ rates:
* a tabular version from {cite:t}`ffn`
* a tabular version from {cite:t}`langanke:2001`
* a simple decay constant from ReacLib

For a network, we would only use one on these.  In fact, we will raise an exception if we try to use them all:

In [10]:
net = pyna.RateCollection(libraries=[new_lib])

[[Ni56 ⟶ Co56 + e⁺ + 𝜈, Ni56 + e⁻ ⟶ Co56 + 𝜈, Ni56 + e⁻ ⟶ Co56 + 𝜈]]


RateDuplicationError: Duplicate rates found

```{tip}
This is one key difference between a `Library` and a network in pynucastro---a `Library` can have duplicate rates, and can be thought of as a container used in the process of selecting the rates that will finally be part of your network.
```